# 1 - Imports básicos

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama.chat_models import ChatOllama
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_ollama.embeddings import OllamaEmbeddings

# 2 - Carregando dados do Wikipedia

In [ ]:
loader = WebBaseLoader(
    "https://pt.wikipedia.org/wiki/Silvio_Santos"
)
docs = loader.load()
print(f"Número de documentos {len(docs)}")

In [ ]:
documento = docs[0]
print(f"Documento: \n\n {documento}")

In [ ]:
print(f"Tipo do objeto documento: {type(documento)}")

document_as_dict = vars(documento)
print(f"Propriedades do documento: {document_as_dict.keys()}")

# 3 - Dividindo dados da página em chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)

print(f"Tipo do objeto chunks (acho que é uma lista): {type(chunks)}")
print(f"Número de chunks: {len(chunks)}")

In [ ]:
print("Vamos dar uma olhada nos chunks... \n\n")

for i in range(20, 25):
    print(f"Chunk #{i}: \n{chunks[i]}\n-------------------------------\n")
# print(chunks[40:45])

# 4 - Criando embeddings para os chunks de texto

Aqui associamos cada pedaço de texto a um vetor multidimensional e criamos uma base de dados semântica, de onde podemos extrair trechos semanticamente relacionados por similaridade de cossenos.

In [ ]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")
Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./data",
)

# 5 - Criando cadeia do LangChain para executar o RAG

In [ ]:
vectorstore = Chroma(persist_directory="./data", embedding_function=embeddings)
retriever = vectorstore.as_retriever()

retriever

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Você é um assistente de IA que responde às perguntas do usuário com base no contexto abaixo:

            <context>
                {context}
            </context>

            Responda às perguntas do usuário utilizando apenas o contexto acima.
            """,
        ),
        ("human", "{question}"),
    ]
)

prompt

In [ ]:
llm = ChatOllama(
        model="llama3.2",
        temperature=0.001
    )

llm

In [ ]:
parser = StrOutputParser()
parser

In [ ]:
# retriever.input_schema.schema()

# prompt.input_schema.schema()

chain = (
    {
        'question': RunnablePassthrough(),
        'context': retriever
    } |  
    prompt |
    llm | 
    parser
)

# chain
chain.input_schema

# 5 - Respondendo a uma pergunta